## Introduction
This sample notebook shows how to log metrics to TensorBoard. For those not familiar with TensorBoard, it is a visualization tool originally developed for TensorFlow but now also very usable for other frameworks like PyTorch. Under the hood, the logger uses the SummaryWriter to write data to disk. This is done by passing a writer object as an argument to the constructor.

For the rest this notebook uses the same dummy data and `resnet18` model as the basic_fos example.
So check that one out if not all is clear.


In [3]:
# Uncomment the following line if running on Google Colab
# !pip install fos

import torch
import torch.nn.functional as F
from torchvision.models import resnet18 
from torch.utils.tensorboard import SummaryWriter

# Import the  Fos classes we'll use
from fos import Workout
from fos.callbacks import NotebookMeter, TensorBoardMeter

## Setup
We first create an instance of the model we want to train, so in this case the resnet18 model. Throughout this example we refer to the instance of model as the `predictor` in order to be able to differentiate from the supervisor we create later on.

After the predictor is instantiated, the optimizer and loss function are created. If you are familiar with PyTorch this should all be straight forward. We choose Adam as the optimizer since it performs well in many scenarios. But feel free to swap it out for any other optimizer. And as a loss function we choose the `binary cross entropy` as that fits the multi-class classification problem well.

And finally time to create some random dummy data that mimics an image of 224x224 pixels and the target: 

     X: 4x3x224x224 = 4 samples in a batch x 3 channels (RGB) x 224 pixels width x 224 pixels height
     
     Y: 4x1000      = 4 samples in a batch x 1000 possible classes 

In a real world scenario's this would typically be implemented as a PyTorch Dataloader. But for the purpose of this notebook a simple list of random tensors will do just fine.

In [4]:
predictor = resnet18()
optim     = torch.optim.Adam(predictor.parameters())
loss      = F.binary_cross_entropy_with_logits

data = [(torch.randn(4,3,224,224), torch.rand(4,1000).round()) for i in range(10)]

## Setup metrics
We create a writer to that will handle creating the nesecarry files and writing the data to these files. The SummaryWriter is part of the excellent TensorBoardX package.

After we have an instance of the writer, we can create a TensorBoardMeter. In this example it will only process and store the `val_loss` metric.

Finally we create the ParamHistogtam metric. This is a model metric, in other words checks the performance of the model and not the performance of the predictions. In this case it will create histograms of the weights and gradients of the layers in the model. We pass the same writer as an argument. You can use a different writer, but then it should also write to a different file.

In [5]:
workout = Workout(predictor, loss, optim)

In [6]:
# Writer will output to ./runs/ directory by default
writer  = SummaryWriter()
nbmeter = NotebookMeter()
tbmeter = TensorBoardMeter(writer)

## Train
Now we train the model. In order to see the tensorbaord output, you'll need to start tensorboard and point it to the directory where this notebook is logging its data. This is the ./runs/ directory by default.

In [7]:
workout.fit(data, epochs=5, callbacks=[nbmeter, tbmeter])

[  1:    10] - loss: 0.7227: 100%|██████████|00:01<00:00
[  2:    20] - loss: 0.6641: 100%|██████████|00:01<00:00
[  3:    30] - loss: 0.6633: 100%|██████████|00:01<00:00
[  4:    40] - loss: 0.6488: 100%|██████████|00:01<00:00
[  5:    50] - loss: 0.5993: 100%|██████████|00:01<00:00
